## Install the Spanner Python API

In [ ]:
! pip install --upgrade google-cloud-spanner

## Set the following variables 

You at least need to change the Project ID variable to your project's ID. You can change the other variables if you like. 

In [ ]:
project_id = 'your-project-id-here'
instance_id = 'spanner-instance-python'

processing_units = 100
database_id = 'pets-db'

OPERATION_TIMEOUT_SECONDS = 240

!gcloud services enable spanner.googleapis.com 
print("Spanner Enabled")

print("Done")

## Create a Spanner instance

The code below uses the Python library to create a Spanner instance. 

In [ ]:
import time
from google.cloud import spanner

def create_instance_with_processing_units(instance_id, processing_units):
    """Creates an instance."""
    spanner_client = spanner.Client(project=project_id)

    config_name = "{}/instanceConfigs/regional-us-central1".format(
        spanner_client.project_name
    )

    instance = spanner_client.instance(
        instance_id,
        configuration_name=config_name,
        display_name=instance_id,
        # Use node_count attribute to create instance using nodes
        # instead of Processing Units
        # node_count = 1,
        processing_units=processing_units,
        labels={
            "created": str(int(time.time())),
        },
    )

    # create() returns a long-running operation
    operation = instance.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print(
        "Created instance {} with {} processing units".format(
            instance_id, instance.processing_units
        )
    )


# Call the function
create_instance_with_processing_units(instance_id, processing_units)

## Go to the Google Cloud Console and verify that the Spanner instance was created. 

## Create the Pets database

This code create the Pets database on the instance created above. Note the DDL code that creates the Owners and Pets tables. 

In [ ]:
def create_database(instance_id, database_id):
    """Creates a database and tables for sample data."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)

    database = instance.database(
        database_id,
        ddl_statements=[
            """CREATE TABLE Owners (
                  OwnerID STRING(36) NOT NULL,
                  OwnerName STRING(MAX) NOT NULL
               ) PRIMARY KEY (OwnerID)""",
            """CREATE TABLE Pets (
                  PetID STRING(36) NOT NULL, 
                  OwnerID STRING(36) NOT NULL, 
                  PetType STRING(MAX) NOT NULL,
                  PetName STRING(MAX) NOT NULL,
                  Breed STRING(MAX) NOT NULL,
              ) PRIMARY KEY (PetID)"""
        ],
    )

    # create() returns a long-running operation
    operation = database.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Created database {} on instance {}".format(database_id, instance_id))

create_database(instance_id, database_id)


## Show the data definition code for the Pets database 

Verify that the DDL code from the last code block was set correctly. This uses the Admin API to retrieve the DDL. 

In [ ]:
def get_database_ddl(instance_id, database_id):
  
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    ddl = spanner_client.database_admin_api.get_database_ddl(database=database.name)

    print("Retrieved database DDL for {}".format(database_id))

    for statement in ddl.statements:
        print(statement)


get_database_ddl(instance_id, database_id)

## Add a few test records

Note the batch insert in this example to add multiple records. We could run an inter-query as well, but this is more efficient when adding multiple records at the same time. 

Also, note the code to create the UUIDs that are used for primary key values. 

In [ ]:
import uuid

def insert_data(instance_id, database_id):
    doug_id = str(uuid.uuid4())
    john_id = str(uuid.uuid4())
    sue_id = str(uuid.uuid4())

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.batch() as batch:
        batch.insert(
            table="Owners",
            columns=("OwnerID", "OwnerName"),
            values=[
                (doug_id, u"Doug"),
                (john_id, u"John"),
                (sue_id, u"Sue"),
            ],
        )

        batch.insert(
            table="Pets",
            columns=("PetID", "OwnerID", "PetType", "PetName", "Breed"),
            values=[
                (str(uuid.uuid4()), doug_id, u"Dog", u"Noir", u"Schnoodle"),
                (str(uuid.uuid4()), doug_id, u"Dog", u"Bree", u"Mutt"),
                (str(uuid.uuid4()), doug_id, u"Cat", u"Tom", u"Alley"),
                (str(uuid.uuid4()), john_id, u"Dog", u"Duke", u"GoldenDoodle"),\
                (str(uuid.uuid4()), john_id, u"Dog", u"Sparky", u"Poodle"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Cuff", u"Box"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Link", u"Box"),
                (str(uuid.uuid4()), sue_id, u"Cat", u"Cleo", u"Domestic"),
            ],
        )

    print("Inserted data.")


insert_data(instance_id, database_id)

## Run a query using SQL

Run a SELECT query to test whether the test records were added. 

In [ ]:
def run_simple_query(sql):
  spanner_client = spanner.Client(project=project_id)
  instance = spanner_client.instance(instance_id)
  database = instance.database(database_id)

  # Execute a simple SQL statement.
  with database.snapshot() as snapshot:
      results = snapshot.execute_sql(sql)
      for row in results:
        print(row)


sql = """SELECT OwnerName, PetName, PetType, Breed 
         FROM Owners 
         JOIN Pets ON Owners.OwnerID = Pets.OwnerID;"""


run_simple_query(sql)

## Read data from a table

The code above runs a SELECT query. In this block, columns are just read from tables. Notice the function here is snapshot.read(), in the prior block it was snapshot.execute_sql().

In [ ]:
def read_data(instance_id, database_id):
    """Reads sample data from the database."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.snapshot() as snapshot:
        keyset = spanner.KeySet(all_=True)
        results = snapshot.read(
            table="Pets", columns=("PetID", "PetName", "PetType", "Breed"), keyset=keyset
        )

        for row in results:
            print(u"PetID: {}, PetName: {}, PetType: {}, Breed: {}".format(*row))


# Call the function
read_data(instance_id, database_id)

## Go to the Console and verify the Spanner instance, database, tables, and data were all created 

## Delete all of the data

Here we just delete all of the data by running a couple of DELETE queries. 


In [ ]:
def delete_data_with_dml(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    def delete_pets_owners(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM Pets WHERE true = true"
        )
        row_ct += transaction.execute_update(
            "DELETE FROM Owners WHERE true = true"
        )

        print("{} record(s) deleted.".format(row_ct))

    database.run_in_transaction(delete_pets_owners)

# Call the function
delete_data_with_dml(instance_id, database_id)


## Delete the database

In this block, the database itself is deleted. We didn't actually have to delete the data prior to deleting the database. 

In [ ]:
def delete_database(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    
    database.drop()

    print("{} database dropped".format(database_id))
 

# Call the function
delete_database(instance_id, database_id)

## Delete the instance

We're done. Let's delete the Spanner instance since it costs money. 

In [ ]:
def delete_spanner_instance(instance_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    instance.delete()

    print("{} instance deleted".format(instance_id))

# Call the function
delete_spanner_instance(instance_id)

## Go to the Console and see if the Spanner instance was deleted. 